# Analise do Algorítmo para cálculo das médias em Dev Médias
## Início

Este projeto visa ter o cálculo, a partir de notas de entrada, juntamente com todos os pesos de todas as notas, e uma média visada, encontrar as melhores notas que um aluno deve tirar para conseguir a tal média esperada. 
Inicialmente, este problema será separado em problemáticas, a fim de compreendê-lo melhor. 


## 0 - Estruturação do problema
Cada nota será separada em uma classe, com peso e valor da nota. 
1

### 0.0 - Importando bibliotecas

In [10]:
from typing import Tuple, List, Dict
import datetime
import abc
import random 

### 0.1 - Definição das variáveis/ constantes

In [11]:
dominio_de_notas = list(map(lambda x: x/2, range(0,21)))


### 0.2 - Regras de negócio
### 0.2.1 - Nota
- O valor da nota pode ser None, mas o peso não;
- Cada nota deve pertencer ao domínio `dominio_de_notas`;
- O peso deve ser menor que 1

In [12]:
class Nota:
    def __init__(self, peso: float = None, nota: float = None):
        nota_valida, msg = self.valida_nota(nota)
        if(not nota_valida): 
            raise Exception(msg)
        self.nota = nota
        
        peso_valido, msg = self.valida_peso(peso)
        if(not peso_valido):
            raise Exception(msg)
        self.peso = peso
        
    @staticmethod
    def valida_nota(nota: int) -> Tuple[bool, str]:
        if(type(nota) not in [float, int, None]): 
            return (False, f"Nota {nota} deve ser um número")
        if(nota not in dominio_de_notas): 
            return (False, f"Nota {nota} deve estar entre 0 e 10, variando de 0.5 em 0.5")
        return (True, '')
        
    @staticmethod
    def valida_peso(peso: int) -> Tuple[bool, str]:
        if(peso is None): 
            return (False, f"Peso {peso} não pode ser nulo")
        elif(type(peso) != float): 
            return (False, f"Peso {peso} deve ser um número")
        elif (peso < 0): 
            return (False, f"Peso {peso} não pode ser menor que 0")
        elif(peso > 1): 
            return (False, f"Peso {peso} não pode ser maior que 1")
        return (True, '')
    
    def __str__(self):
        return f"(Valor: {self.nota}, Peso: {self.peso})"

Testando classe `Nota`

In [13]:
# Sucesso
nota = Nota(peso=0.5, nota=10)
print("Teste - Sucesso")
print(nota.nota == 10)
print(nota.peso == 0.5)

# Nota deve ser um número
print("\nTeste - Nota deve ser um número")
try:
    nota = Nota(peso=0.5, nota='10')
    print(False)
except Exception as e:
    print(e.args[0] == "Nota 10 deve ser um número")
    
# Nota deve estar entre 0 e 10, variando de 0.5 em 0.5
print("\nTeste - Nota deve estar entre 0 e 10, variando de 0.5 em 0.5")
try:
    nota = Nota(peso=0.5, nota=11)
    print(False)
except Exception as e:
    print(e.args[0] == "Nota 11 deve estar entre 0 e 10, variando de 0.5 em 0.5")
    
# Peso não pode ser nulo
print("\nTeste - Peso não pode ser nulo")
try:
    nota = Nota(nota=10)
    print(False)
except Exception as e:
    print(e.args[0] == "Peso None não pode ser nulo")
    
# Peso deve ser um número
print("\nTeste - Peso deve ser um número")
try:
    nota = Nota(peso='0.5', nota=10)
    print(False)
except Exception as e:
    print(e.args[0] == "Peso 0.5 deve ser um número")
    
# Peso não pode ser menor que 0
print("\nTeste - Peso não pode ser menor que 0")
try:
    nota = Nota(peso=-0.5, nota=10)
    print(False)
except Exception as e:
    print(e.args[0] == "Peso -0.5 não pode ser menor que 0")
    
# Peso não pode ser maior que 1
print("\nTeste - Peso não pode ser maior que 1")
try:
    nota = Nota(peso=1.5, nota=10)
    print(False)
except Exception as e:
    print(e.args[0] == "Peso 1.5 não pode ser maior que 1")



Teste - Sucesso
True
True

Teste - Nota deve ser um número
True

Teste - Nota deve estar entre 0 e 10, variando de 0.5 em 0.5
True

Teste - Peso não pode ser nulo
True

Teste - Peso deve ser um número
True

Teste - Peso não pode ser menor que 0
True

Teste - Peso não pode ser maior que 1
True


#### 0.2.2 - Utils
Funções que serão utilizadas com frequência


In [102]:
class Utils:
    
    @staticmethod
    def print_lista_de_notas(l: List[Nota]):
        if(len(l) == 0):
            print("[]")
            return
        print("[", end=' ')
        for idx in range(len(l)-1):
            print(l[idx], end=', ')
        print(l[-1], end=' ]')

    @staticmethod
    def media_aritimetica(l: List[float]) -> float:
        return sum(l)/len(l)
        
    @staticmethod
    def media(l: List[Nota]) -> float:
        if sum(map(lambda x: x.peso, l)) != 1:
            raise Exception("A soma dos pesos deve ser 1")
        return sum(map(lambda x: x.nota * x.peso, l))

    @staticmethod
    def desvio_padrao(l: List[float]) -> float:
        media = Utils.media_aritimetica(l)
        return (sum(map(lambda x: (x - media)**2, l))/(len(l)-1))**(1/2)
    
    @staticmethod
    def distancia_entre_notas(l: List[float], distance_min: float) -> bool:
        media = Utils.media_aritimetica(l)
        return all(map(lambda x: abs(x - media) <= distance_min, l))

Testando funções de `Utils`

In [107]:
# Media - Teste 1
print("\nTeste - Media - Teste 1")
notas = [Nota(peso=0.5, nota=10), Nota(peso=0.5, nota=10)]
print(Utils.media(notas) == 10)

# Media - Teste 2
print("\nTeste - Media - Teste 2")
notas = [Nota(peso=0.5, nota=10), Nota(peso=0.5, nota=0)]
print(Utils.media(notas) == 5)

# Media - Teste 3
print("\nTeste - Media - Teste 3")
notas = [Nota(peso=0.5, nota=10), Nota(peso=0.5, nota=5)]
print(Utils.media(notas) == 7.5)

# Media - Teste 4
print("\nTeste - Media - Teste 4")
notas = [Nota(peso=0.5, nota=10), Nota(peso=0.3, nota=5), Nota(peso=0.2, nota=0)]
print(Utils.media(notas) == 6.5)

# Media - Teste do Erro
print("\nTeste - Media - Teste do Erro")
notas = [Nota(peso=0.5, nota=10), Nota(peso=0.5, nota=5), Nota(peso=0.5, nota=0), Nota(peso=0.5, nota=0)]
try:
    Utils.media(notas)
    print(False)
except Exception as e:
    print(e.args[0] == "A soma dos pesos deve ser 1")

# Desvio Padrão - Teste 1
print("\nTeste - Desvio Padrão - Teste 1")
notas = [Nota(peso=0.5, nota=10), Nota(peso=0.5, nota=10)]
print(Utils.desvio_padrao([x.nota for x in notas]) == 0)

# Desvio Padrão - Teste 2
print("\nTeste - Desvio Padrão - Teste 2")
notas = [Nota(peso=0.5, nota=10), Nota(peso=0.5, nota=0)]
print(7.0 < Utils.desvio_padrao([x.nota for x in notas]) < 7.1)

# Desvio Padrão - Teste 3
print("\nTeste - Desvio Padrão - Teste 3")
notas = [Nota(peso=0.25, nota=10), Nota(peso=0.3, nota=5), Nota(peso=0.2, nota=6), Nota(peso=0.25, nota=6)]
print(2.2 < Utils.desvio_padrao([x.nota for x in notas]) < 2.3)

# Distancia entre notas - Teste 1
print("\nTeste - Distancia entre notas - Teste 1")
notas = [Nota(peso=0.5, nota=10), Nota(peso=0.5, nota=10)]
print(Utils.distancia_entre_notas([x.nota for x in notas], 0.5) == True)

# Distancia entre notas - Teste 2
print("\nTeste - Distancia entre notas - Teste 2")
notas = [Nota(peso=0.5, nota=10), Nota(peso=0.5, nota=0)]
print(Utils.distancia_entre_notas([x.nota for x in notas], 0.5) == False)

# Distancia entre notas - Teste 3
print("\nTeste - Distancia entre notas - Teste 3")
notas = [Nota(peso=0.25, nota=7), Nota(peso=0.3, nota=5), Nota(peso=0.2, nota=6), Nota(peso=0.25, nota=6)]
print(Utils.distancia_entre_notas([x.nota for x in notas], 3) == True)


Teste - Media - Teste 1
True

Teste - Media - Teste 2
True

Teste - Media - Teste 3
True

Teste - Media - Teste 4
True

Teste - Media - Teste do Erro
True

Teste - Desvio Padrão - Teste 1
True

Teste - Desvio Padrão - Teste 2
True

Teste - Desvio Padrão - Teste 3
True

Teste - Distancia entre notas - Teste 1
True

Teste - Distancia entre notas - Teste 2
True

Teste - Distancia entre notas - Teste 3
True


#### 0.2.3 - Solucionador
Interface para classes que resolverão os problemas abaixo

In [16]:
class Solucionador(abc.ABC):
    
    @abc.abstractmethod
    def algoritmo(self, notas_que_tenho: List[Tuple[int, int]], peso_de_notas_que_quero: List[float], media_desejada: float) -> List[Nota]:
        pass
    
    @abc.abstractmethod
    def teste_algoritmo(self, notas_que_tenho: Dict[float, float], peso_de_notas_que_quero: List[float], media_desejada: float) -> None:
        pass

#### 0.2.4 - Observações de regras de negócio do algoritmo

- *exemplo explicativo:* se o aluno tirar entre 4.96 - 4.99 de média, ele terá média final 5.0;
- se não existir uma combinação de na qual o aluno consegue tirar a média certinha, o algoritmo deve responder com alguma alteração;
- se for impossível do aluno tirar uma média (*ex:* quer tirar média 10, mas já tirou 0 em alguma média), o algoritmo deve notificar;

## 1. Primeiro Caso
Supondo que uma matéria tenha P1, P2, P3 e P4, sendo que já foram dadas as notas da P1 e P2, quero saber para quais notas P3 e P4 o aluno conseguirá média 6.
Facilitadores:
1. tamanho de notas fixo
2. poucas notas
3. nota possível de ser determinada

In [17]:
class PrimeiroCaso(Solucionador):
    @staticmethod
    def algoritmo(notas_que_tenho: List[Tuple[int, int]], peso_de_notas_que_quero: List[float], media_desejada: float) -> List[Nota]:
        notas_possiveis = list()
        for i in range(len(dominio_de_notas)):
            for j in range(len(dominio_de_notas)):
                if(Utils.media(
                    [Nota(nota[0], nota[1]) for nota in notas_que_tenho] + \
                    [
                        Nota(peso=peso_de_notas_que_quero[0], nota=dominio_de_notas[i]),
                        Nota(peso=peso_de_notas_que_quero[1], nota=dominio_de_notas[j])
                    ]
                )) == media_desejada:
                    notas_possiveis.append((dominio_de_notas[i], dominio_de_notas[j]))
        return notas_possiveis
    
    @staticmethod
    def teste_algoritmo(notas_que_tenho: Dict[float, float], peso_de_notas_que_quero: List[float], media_desejada: float) -> None:
        notas_possiveis = PrimeiroCaso.algoritmo(notas_que_tenho=notas_que_tenho, peso_de_notas_que_quero=peso_de_notas_que_quero, media_desejada=media_desejada)
        print(f"Para as notas {notas_que_tenho}, pesos {peso_de_notas_que_quero} e média {media_desejada} as notas possíveis são:")
        for notas in notas_possiveis:
            print(f"{notas[0]} e {notas[1]}")

In [18]:
# Teste 1
print("\nPrimeiroCaso - Teste 1")
P1 = Nota(peso=0.2, nota=6.0)
P2 = Nota(peso=0.2, nota=5.0)
P3_peso = 0.4
P4_peso = 0.2
media_desejada = 6

PrimeiroCaso.teste_algoritmo(notas_que_tenho=[(P1.peso, P1.nota), (P2.peso, P2.nota)], peso_de_notas_que_quero=[P3_peso, P4_peso], media_desejada=media_desejada)

# Verificação
print("\n"+
str(Utils.media([
    P1,
    P2,
    Nota(peso=P3_peso, nota=5.0),
    Nota(peso=P4_peso, nota=8.0)
]) == media_desejada)
)

# Teste 2
print("\nPrimeiroCaso - Teste 2")
P1 = Nota(peso=0.2, nota=7.0)
P2 = Nota(peso=0.2, nota=7.0)
P3_peso = 0.3
P4_peso = 0.3
media_desejada = 7

PrimeiroCaso.teste_algoritmo(notas_que_tenho=[(P1.peso, P1.nota), (P2.peso, P2.nota)], peso_de_notas_que_quero=[P3_peso, P4_peso], media_desejada=media_desejada)

# Verificação
print("\n"+
str(Utils.media([
    P1,
    P2,
    Nota(peso=P3_peso, nota=6.5),
    Nota(peso=P4_peso, nota=7.5)
])==media_desejada)
)




PrimeiroCaso - Teste 1
Para as notas [(0.2, 6.0), (0.2, 5.0)], pesos [0.4, 0.2] e média 6 as notas possíveis são:
4.5 e 10.0
5.0 e 9.0
5.5 e 8.0
7.0 e 5.0
7.5 e 4.0
8.0 e 3.0
9.5 e 0.0

False

PrimeiroCaso - Teste 2
Para as notas [(0.2, 7.0), (0.2, 7.0)], pesos [0.3, 0.3] e média 7 as notas possíveis são:
4.0 e 10.0
4.5 e 9.5
5.0 e 9.0
5.5 e 8.5
6.0 e 8.0
6.5 e 7.5
7.0 e 7.0
8.0 e 6.0
8.5 e 5.5
9.0 e 5.0
9.5 e 4.5

True


**Observação**: Poucas combinações de `notas_que_tenho` e `media_desejada` calculam o que quero...

## 2. Segundo Caso
Supondo que uma matéria tenha P1, P2, P3 e P4, sendo que já foram dadas as notas da P1, T1 e T2, quero saber para quais notas P2, P3, P4, T3 e T4 o aluno conseguirá média 7.
Facilitadores:
1. tamanho de notas fixo
2. nota possível de ser determinada

In [19]:
class SegundoCaso(Solucionador):
    @staticmethod
    def algoritmo(notas_que_tenho: List[Nota], peso_de_notas_que_quero: List[float], media_desejada: float) -> List[Nota]:
        notas_possiveis = list()
        for i in range(len(dominio_de_notas)):
            for j in range(len(dominio_de_notas)):
                for k in range(len(dominio_de_notas)):
                    for l in range(len(dominio_de_notas)):
                        for m in range(len(dominio_de_notas)):
                            if(Utils.media(
                                notas_que_tenho + \
                                [
                                    Nota(peso=peso_de_notas_que_quero[0], nota=dominio_de_notas[i]),
                                    Nota(peso=peso_de_notas_que_quero[1], nota=dominio_de_notas[j]),
                                    Nota(peso=peso_de_notas_que_quero[2], nota=dominio_de_notas[k]),
                                    Nota(peso=peso_de_notas_que_quero[3], nota=dominio_de_notas[l]),
                                    Nota(peso=peso_de_notas_que_quero[4], nota=dominio_de_notas[m])
                                ]
                            ) == media_desejada):
                                notas_possiveis.append((dominio_de_notas[i], dominio_de_notas[j], dominio_de_notas[k], dominio_de_notas[l], dominio_de_notas[m]))
        return notas_possiveis
    
    @staticmethod
    def teste_algoritmo(notas_que_tenho: Dict[float, float], peso_de_notas_que_quero: List[float], media_desejada: float) -> None:
        notas_possiveis = SegundoCaso.algoritmo(notas_que_tenho=notas_que_tenho, peso_de_notas_que_quero=peso_de_notas_que_quero, media_desejada=media_desejada)
        print("Para as notas:") 
        Utils.print_lista_de_notas(notas_que_tenho)
        print(f", pesos {peso_de_notas_que_quero} e média {media_desejada} as notas possíveis são:")
        Utils.print_lista_de_notas(notas_possiveis)

Para 5 notas o tempo está de mais ou menos 27s ...
Talvez escolhendo apenas 3 notas possa ajudar ...

Outro ponto a se observar é que também tem casos em que não existem notas possíveis de se determinar as médias...

## 3. Terceiro Caso
Supondo que uma matéria tenha P1, P2, P3 e P4, sendo que já foram dadas as notas da P1, T1 e T2, quero saber para quais notas P2, P3, P4, T3 e T4 o aluno conseguirá média 7.
Facilitadores:
1. tamanho de notas fixo
2. nota possível de ser determinada

Adicionais:
1. o programa parará de rodar ao escolher 3 notas possíveis

In [20]:
class TerceiroCaso(Solucionador):
    @staticmethod
    def algoritmo(notas_que_tenho: List[Nota], peso_de_notas_que_quero: List[float], media_desejada: float) -> List[Nota]:
        notas_possiveis = list()
        for i in range(len(dominio_de_notas)):
            for j in range(len(dominio_de_notas)):
                for k in range(len(dominio_de_notas)):
                    for l in range(len(dominio_de_notas)):
                        for m in range(len(dominio_de_notas)):
                            if(Utils.media(
                                notas_que_tenho + \
                                [
                                    Nota(peso=peso_de_notas_que_quero[0], nota=dominio_de_notas[i]),
                                    Nota(peso=peso_de_notas_que_quero[1], nota=dominio_de_notas[j]),
                                    Nota(peso=peso_de_notas_que_quero[2], nota=dominio_de_notas[k]),
                                    Nota(peso=peso_de_notas_que_quero[3], nota=dominio_de_notas[l]),
                                    Nota(peso=peso_de_notas_que_quero[4], nota=dominio_de_notas[m])
                                ]
                            ) == media_desejada):
                                notas_possiveis.append((dominio_de_notas[i], dominio_de_notas[j], dominio_de_notas[k], dominio_de_notas[l], dominio_de_notas[m]))
                            if(len(notas_possiveis) == 3): return notas_possiveis
        return notas_possiveis
    
    @staticmethod
    def teste_algoritmo(notas_que_tenho: Dict[float, float], peso_de_notas_que_quero: List[float], media_desejada: float) -> None:
        notas_possiveis = TerceiroCaso.algoritmo(notas_que_tenho=notas_que_tenho, peso_de_notas_que_quero=peso_de_notas_que_quero, media_desejada=media_desejada)
        print("Para as notas:") 
        Utils.print_lista_de_notas(notas_que_tenho)
        print(f", pesos {peso_de_notas_que_quero} e média {media_desejada} as notas possíveis são:")
        Utils.print_lista_de_notas(notas_possiveis)

In [21]:
# Teste 1
print("\nTerceiroCaso - Teste 1")
P1 = Nota(peso=0.2*0.6, nota=6.0)
T1 = Nota(peso=0.2*0.4, nota=6.0)

P2_peso = 0.2*0.6
T2 = Nota(peso=0.2*0.4, nota=6.0)

P3_peso = 0.3*0.6
T3_peso = 0.3*0.4

P4_peso = 0.3*0.6
T4_peso = 0.3*0.4

media_desejada = 6

l_notas_que_tenho = [P1, T1, T2]

TerceiroCaso.teste_algoritmo(notas_que_tenho=l_notas_que_tenho, peso_de_notas_que_quero=[P2_peso, T3_peso, P3_peso, T4_peso, P4_peso], media_desejada=media_desejada)


TerceiroCaso - Teste 1
Para as notas:
[ (Valor: 6.0, Peso: 0.12), (Valor: 6.0, Peso: 0.08000000000000002), (Valor: 6.0, Peso: 0.08000000000000002) ], pesos [0.12, 0.12, 0.18, 0.12, 0.18] e média 6 as notas possíveis são:
[ (0.0, 0.0, 8.0, 9.0, 10.0), (0.0, 0.0, 8.5, 9.0, 9.5), (0.0, 0.0, 9.0, 9.0, 9.0) ]

Agora o algorítmo está com 0.2 s de duração...

Porém,  ele escolheu notas de grande esforço para tirar.

## 4. Quarto Caso
Supondo que uma matéria tenha P1, P2, P3 e P4, sendo que já foram dadas as notas da P1, T1 e T2, quero saber para quais notas P2, P3, P4, T3 e T4 o aluno conseguirá média 7.
Facilitadores:
1. tamanho de notas fixo
2. nota possível de ser determinada

Adicionais:
1. o programa parará de rodar ao escolher 3 notas possíveis
2. a ordem da permutação será aleatória

In [89]:
class QuartoCaso(Solucionador):
    @staticmethod
    def algoritmo(notas_que_tenho: List[Nota], peso_de_notas_que_quero: List[float], media_desejada: float) -> List[Nota]:
        notas_possiveis = list()
        dominio_i = dominio_de_notas.copy()
        random.shuffle(dominio_i)
        for i in range(len(dominio_de_notas)):
            dominio_j = dominio_de_notas.copy()
            random.shuffle(dominio_j)
            for j in range(len(dominio_de_notas)):
                dominio_k = dominio_de_notas.copy()
                random.shuffle(dominio_k)                
                for k in range(len(dominio_de_notas)):
                    dominio_l = dominio_de_notas.copy()
                    random.shuffle(dominio_l)
                    for l in range(len(dominio_de_notas)):
                        dominio_m = dominio_de_notas.copy()
                        random.shuffle(dominio_m)
                        for m in range(len(dominio_de_notas)):
                            if(Utils.media(
                                notas_que_tenho + \
                                [
                                    Nota(peso=peso_de_notas_que_quero[0], nota=dominio_i[i]),
                                    Nota(peso=peso_de_notas_que_quero[1], nota=dominio_j[j]),
                                    Nota(peso=peso_de_notas_que_quero[2], nota=dominio_k[k]),
                                    Nota(peso=peso_de_notas_que_quero[3], nota=dominio_l[l]),
                                    Nota(peso=peso_de_notas_que_quero[4], nota=dominio_m[m])
                                ]
                            ) == media_desejada):
                                notas_possiveis.append((dominio_i[i], dominio_j[j], dominio_k[k], dominio_l[l], dominio_m[m]))
                            if(len(notas_possiveis) == 3): return notas_possiveis
        return notas_possiveis
    
    @staticmethod
    def teste_algoritmo(notas_que_tenho: Dict[float, float], peso_de_notas_que_quero: List[float], media_desejada: float) -> None:
        notas_possiveis = QuartoCaso.algoritmo(notas_que_tenho=notas_que_tenho, peso_de_notas_que_quero=peso_de_notas_que_quero, media_desejada=media_desejada)
        print("Para as notas:") 
        Utils.print_lista_de_notas(notas_que_tenho)
        print(f", pesos {peso_de_notas_que_quero} e média {media_desejada} as notas possíveis são:")
        Utils.print_lista_de_notas(notas_possiveis)

In [91]:
# Teste 1
print("QuartoCaso - Teste 1")
P1 = Nota(peso=0.2*0.6, nota=6.0)
T1 = Nota(peso=0.2*0.4, nota=6.0)

P2_peso = 0.2*0.6
T2 = Nota(peso=0.2*0.4, nota=6.0)

P3_peso = 0.3*0.6
T3_peso = 0.3*0.4

P4_peso = 0.3*0.6
T4_peso = 0.3*0.4

media_desejada = 6

l_notas_que_tenho = [P1, T1, T2]

QuartoCaso.teste_algoritmo(notas_que_tenho=l_notas_que_tenho, peso_de_notas_que_quero=[P2_peso, T3_peso, P3_peso, T4_peso, P4_peso], media_desejada=media_desejada)

QuartoCaso - Teste 1
Para as notas:
[ (Valor: 6.0, Peso: 0.12), (Valor: 6.0, Peso: 0.08000000000000002), (Valor: 6.0, Peso: 0.08000000000000002) ], pesos [0.12, 0.12, 0.18, 0.12, 0.18] e média 6 as notas possíveis são:
[ (5.0, 5.0, 3.5, 9.5, 7.5), (5.0, 5.0, 3.5, 6.5, 9.5), (5.0, 5.0, 2.0, 8.0, 10.0) ]

1. Se não tem notas que sua combinação dêem a média pedida, o tempo de espera é enorme
2. Se for possível uma combinação de notas, mesmo assim tem chance das notas não possuirem um bom "mínimo esforço"
3. O programa exibe o valor do peso de forma estranha: `Peso: 0.08000000000000002`

## 5. Quinto Caso
Supondo que uma matéria tenha P1, P2, P3 e P4, sendo que já foram dadas as notas da P1, T1 e T2, quero saber para quais notas P2, P3, P4, T3 e T4 o aluno conseguirá média 7.
Facilitadores:
1. tamanho de notas fixo
2. nota possível de ser determinada

Adicionais:
1. o programa parará de rodar ao escolher 3 notas possíveis
2. a ordem da permutação será aleatória
3. o algorítmo deve escolher notas com, no máximo, 3 de desvio padrão
4. a média pode variar do valor escolhido para `0.04` abaixo do esperado (para aceitar aredondamentos). *Ex*: se a média desejada for `6.0`, a combinação aceita pode ter média final entre `5.96` e `6.0`.

In [98]:
class QuintoCaso(Solucionador):
    NOTAS_TOTAIS=3
    MIN_DESV_PAD=3

    @staticmethod
    def algoritmo(notas_que_tenho: List[Nota], peso_de_notas_que_quero: List[float], media_desejada: float) -> List[Nota]:
        notas_possiveis = list()
        dominio_i = dominio_de_notas.copy()
        random.shuffle(dominio_i)
        for i in range(len(dominio_de_notas)):
            dominio_j = dominio_de_notas.copy()
            random.shuffle(dominio_j)
            for j in range(len(dominio_de_notas)):
                dominio_k = dominio_de_notas.copy()
                random.shuffle(dominio_k)                
                for k in range(len(dominio_de_notas)):
                    dominio_l = dominio_de_notas.copy()
                    random.shuffle(dominio_l)
                    for l in range(len(dominio_de_notas)):
                        dominio_m = dominio_de_notas.copy()
                        random.shuffle(dominio_m)
                        for m in range(len(dominio_de_notas)):
                            possiveis_notas_que_quero = [
                                    Nota(peso=peso_de_notas_que_quero[0], nota=dominio_i[i]),
                                    Nota(peso=peso_de_notas_que_quero[1], nota=dominio_j[j]),
                                    Nota(peso=peso_de_notas_que_quero[2], nota=dominio_k[k]),
                                    Nota(peso=peso_de_notas_que_quero[3], nota=dominio_l[l]),
                                    Nota(peso=peso_de_notas_que_quero[4], nota=dominio_m[m])
                                ]
                            todas_as_notas = notas_que_tenho + possiveis_notas_que_quero
                            media = Utils.media(todas_as_notas)
                            if(abs(media - media_desejada) <= 0.04):
                                if(Utils.desvio_padrao([nota.nota for nota in possiveis_notas_que_quero]) < QuintoCaso.MIN_DESV_PAD):
                                    notas_possiveis.append((dominio_i[i], dominio_j[j], dominio_k[k], dominio_l[l], dominio_m[m]))
                            if(len(notas_possiveis) == QuintoCaso.NOTAS_TOTAIS): return notas_possiveis
        return notas_possiveis
    
    @staticmethod
    def teste_algoritmo(notas_que_tenho: Dict[float, float], peso_de_notas_que_quero: List[float], media_desejada: float) -> None:
        notas_possiveis = QuintoCaso.algoritmo(notas_que_tenho=notas_que_tenho, peso_de_notas_que_quero=peso_de_notas_que_quero, media_desejada=media_desejada)
        print("Para as notas:") 
        Utils.print_lista_de_notas(notas_que_tenho)
        print(f", pesos {peso_de_notas_que_quero} e média {media_desejada} as notas possíveis são:")
        Utils.print_lista_de_notas(notas_possiveis)

In [100]:
# Teste 1
print("\nQuintoCaso - Teste 1")
P1 = Nota(peso=0.2*0.6, nota=6.0)
T1 = Nota(peso=0.08, nota=6.0)

P2_peso = 0.2*0.6
T2 = Nota(peso=0.08, nota=6.0)

P3_peso = 0.3*0.6
T3_peso = 0.3*0.4

P4_peso = 0.3*0.6
T4_peso = 0.3*0.4

media_desejada = 6

l_notas_que_tenho = [P1, T1, T2]

QuintoCaso.teste_algoritmo(notas_que_tenho=l_notas_que_tenho, peso_de_notas_que_quero=[P2_peso, T3_peso, P3_peso, T4_peso, P4_peso], media_desejada=media_desejada)


QuartoCaso - Teste 1
Para as notas:
[ (Valor: 6.0, Peso: 0.12), (Valor: 6.0, Peso: 0.08), (Valor: 6.0, Peso: 0.08) ], pesos [0.12, 0.12, 0.18, 0.12, 0.18] e média 6 as notas possíveis são:
[ (4.0, 8.5, 9.0, 4.0, 4.0), (4.0, 8.5, 9.0, 6.5, 2.5), (4.0, 8.5, 9.0, 6.0, 2.5) ]

1. O resultado melhorou, mas ainda as notas não parecem ser tão boas a fim se serem consideradas como "esforço mínimo"
2. talvez a ideia de "desvio padrão" não seja tão boa, se tentará criar um algorítmo que verifica se existem quaisquer valores que se distanciam de um valor fixo da média entre eles

## 6. Sexto Caso
Supondo que uma matéria tenha P1, P2, P3 e P4, sendo que já foram dadas as notas da P1, T1 e T2, quero saber para quais notas P2, P3, P4, T3 e T4 o aluno conseguirá média 7.
Facilitadores:
1. tamanho de notas fixo
2. nota possível de ser determinada

Adicionais:
1. o programa parará de rodar ao escolher 3 notas possíveis
2. a ordem da permutação será aleatória
3. o algorítmo deve escolher notas com, no máximo, 3 de distância entre as notas e a média aritimética entre elas
4. a média pode variar do valor escolhido para `0.04` abaixo do esperado (para aceitar aredondamentos). *Ex*: se a média desejada for `6.0`, a combinação aceita pode ter média final entre `5.96` e `6.0`.

In [112]:
class SextoCaso(Solucionador):
    NOTAS_TOTAIS=3
    MIN_DIST=3

    @staticmethod
    def algoritmo(notas_que_tenho: List[Nota], peso_de_notas_que_quero: List[float], media_desejada: float) -> List[Nota]:
        notas_possiveis = list()
        dominio_i = dominio_de_notas.copy()
        random.shuffle(dominio_i)
        for i in range(len(dominio_de_notas)):
            dominio_j = dominio_de_notas.copy()
            random.shuffle(dominio_j)
            for j in range(len(dominio_de_notas)):
                dominio_k = dominio_de_notas.copy()
                random.shuffle(dominio_k)                
                for k in range(len(dominio_de_notas)):
                    dominio_l = dominio_de_notas.copy()
                    random.shuffle(dominio_l)
                    for l in range(len(dominio_de_notas)):
                        dominio_m = dominio_de_notas.copy()
                        random.shuffle(dominio_m)
                        for m in range(len(dominio_de_notas)):
                            possiveis_notas_que_quero = [
                                    Nota(peso=peso_de_notas_que_quero[0], nota=dominio_i[i]),
                                    Nota(peso=peso_de_notas_que_quero[1], nota=dominio_j[j]),
                                    Nota(peso=peso_de_notas_que_quero[2], nota=dominio_k[k]),
                                    Nota(peso=peso_de_notas_que_quero[3], nota=dominio_l[l]),
                                    Nota(peso=peso_de_notas_que_quero[4], nota=dominio_m[m])
                                ]
                            todas_as_notas = notas_que_tenho + possiveis_notas_que_quero
                            media = Utils.media(todas_as_notas)
                            if(abs(media - media_desejada) <= 0.04):
                                if(Utils.distancia_entre_notas([nota.nota for nota in possiveis_notas_que_quero], SextoCaso.MIN_DIST)):
                                    notas_possiveis.append((dominio_i[i], dominio_j[j], dominio_k[k], dominio_l[l], dominio_m[m]))
                            if(len(notas_possiveis) == SextoCaso.NOTAS_TOTAIS): return notas_possiveis
        return notas_possiveis
    
    @staticmethod
    def teste_algoritmo(notas_que_tenho: Dict[float, float], peso_de_notas_que_quero: List[float], media_desejada: float) -> None:
        notas_possiveis = SextoCaso.algoritmo(notas_que_tenho=notas_que_tenho, peso_de_notas_que_quero=peso_de_notas_que_quero, media_desejada=media_desejada)
        print("Para as notas:") 
        Utils.print_lista_de_notas(notas_que_tenho)
        print(f", pesos {peso_de_notas_que_quero} e média {media_desejada} as notas possíveis são:")
        Utils.print_lista_de_notas(notas_possiveis)

In [118]:
# Teste 1
print("\nSextoCaso - Teste 1")
P1 = Nota(peso=0.2*0.6, nota=6.0)
T1 = Nota(peso=0.08, nota=6.0)

P2_peso = 0.2*0.6
T2 = Nota(peso=0.08, nota=6.0)

P3_peso = 0.3*0.6
T3_peso = 0.3*0.4

P4_peso = 0.3*0.6
T4_peso = 0.3*0.4

media_desejada = 6

l_notas_que_tenho = [P1, T1, T2]

SextoCaso.teste_algoritmo(notas_que_tenho=l_notas_que_tenho, peso_de_notas_que_quero=[P2_peso, T3_peso, P3_peso, T4_peso, P4_peso], media_desejada=media_desejada)


SextoCaso - Teste 1
Para as notas:
[ (Valor: 6.0, Peso: 0.12), (Valor: 6.0, Peso: 0.08), (Valor: 6.0, Peso: 0.08) ], pesos [0.12, 0.12, 0.18, 0.12, 0.18] e média 6 as notas possíveis são:
[ (6.5, 6.0, 8.0, 6.5, 3.5), (6.5, 6.0, 8.0, 5.5, 4.0), (6.5, 6.0, 8.0, 6.0, 3.5) ]

Melhorou bastante a escolha entre as notas. Se tentará resolver os outros pontos que não foram explorados até agora.

## 7. Sétimo Caso
Supondo que tenha duas matérias, tal que elas não possuam os mesmos tamanhos de notas necessárias, quero saber para quais notas um aluno que tirou certas notas iniciais conseguirá tirar uma certa média pedida.
Facilitadores:
1. nota possível de ser determinada

Novidade:
1. o programa deve aceitar qualquer tamanho de notas que devem ser determinadas, assim como qualquer tamanho de notas de input
2. o programa parará de rodar ao escolher 3 notas possíveis
3. a ordem da permutação será aleatória

Deve ser adicionado:
1. o algorítmo deve escolher notas com, no máximo, 3 de distância entre as notas e a média aritimética entre elas
2. a média pode variar do valor escolhido para `0.04` abaixo do esperado (para aceitar aredondamentos). *Ex*: se a média desejada for `6.0`, a combinação aceita pode ter média final entre `5.96` e `6.0`.

In [157]:
class SetimoCaso(Solucionador):
    NOTAS_TOTAIS=3

    @staticmethod
    def algoritmo(notas_que_tenho: List[Nota], peso_de_notas_que_quero: List[float], media_desejada: float) -> List[Nota]:
        notas_possiveis = list()
        dominio_de_possiveis_notas = list()
        for _ in range(len(peso_de_notas_que_quero)):
            dominio_de_uma_nota = dominio_de_notas.copy()
            random.shuffle(dominio_de_uma_nota)
            dominio_de_possiveis_notas.append(dominio_de_uma_nota)
        idx_possiveis_notas = [0 for _ in range(len(peso_de_notas_que_quero))]
        
        while(idx_possiveis_notas != [19 for _ in range(len(peso_de_notas_que_quero))]):
            possiveis_notas_que_quero = [Nota(peso=peso_de_notas_que_quero[i], nota=dominio_de_possiveis_notas[i][idx_possiveis_notas[i]]) for i in range(len(peso_de_notas_que_quero))]
            todas_as_notas = notas_que_tenho + possiveis_notas_que_quero
            media = Utils.media(todas_as_notas)

            if(media == media_desejada):
                notas_possiveis.append(tuple([nota.nota for nota in possiveis_notas_que_quero]))
            
            if(len(notas_possiveis) == SetimoCaso.NOTAS_TOTAIS): return notas_possiveis

            idx_possiveis_notas[-1] += 1
            idx_possiveis_notas_invertido = idx_possiveis_notas.copy()[::-1]
            for idx in range(0, len(idx_possiveis_notas)-1):
                if(idx_possiveis_notas_invertido[idx]) == 20:
                    idx_possiveis_notas_invertido[idx] = 0
                    idx_possiveis_notas_invertido[idx+1] += 1
                else:
                    break
                
            idx_possiveis_notas = idx_possiveis_notas_invertido[::-1]

        return notas_possiveis
    
    @staticmethod
    def teste_algoritmo(notas_que_tenho: Dict[float, float], peso_de_notas_que_quero: List[float], media_desejada: float) -> None:
        notas_possiveis = SetimoCaso.algoritmo(notas_que_tenho=notas_que_tenho, peso_de_notas_que_quero=peso_de_notas_que_quero, media_desejada=media_desejada)
        print("Para as notas:") 
        Utils.print_lista_de_notas(notas_que_tenho)
        print(f", pesos {peso_de_notas_que_quero} e média {media_desejada} as notas possíveis são:")
        Utils.print_lista_de_notas(notas_possiveis)
        print        

In [170]:
# Teste 1
print("\nSetimoCaso - Teste 1")
P1 = Nota(peso=0.2*0.6, nota=6.0)
T1 = Nota(peso=0.08, nota=6.0)

P2_peso = 0.2*0.6
T2 = Nota(peso=0.08, nota=6.0)

P3_peso = 0.3*0.6
T3_peso = 0.3*0.4

P4_peso = 0.3*0.6
T4_peso = 0.3*0.4

media_desejada = 6

l_notas_que_tenho = [P1, T1, T2]

SetimoCaso.teste_algoritmo(notas_que_tenho=l_notas_que_tenho, peso_de_notas_que_quero=[P2_peso, T3_peso, P3_peso, T4_peso, P4_peso], media_desejada=media_desejada)

# Verificação
notas = [
    P1,
    T1,
    T2,
    Nota(peso=P2_peso, nota=2.5),
    Nota(peso=T3_peso, nota=2),
    Nota(peso=P3_peso, nota=6),
    Nota(peso=T4_peso, nota=9),
    Nota(peso=P4_peso, nota=9)
]

print("\n",Utils.media(notas) == 6.0)

# Teste 2
print("\nSetimoCaso - Teste 2")
P1 = Nota(peso=0.2, nota=7.0)
P2 = Nota(peso=0.2, nota=7.0)
P3_peso = 0.3
P4_peso = 0.3
media_desejada = 7

l_notas_que_tenho = [P1, P2]

SetimoCaso.teste_algoritmo(notas_que_tenho=l_notas_que_tenho, peso_de_notas_que_quero=[P3_peso, P4_peso], media_desejada=media_desejada)

# Verificação
print("\n"+
str(Utils.media([
    P1,
    P2,
    Nota(peso=P3_peso, nota=9.0),
    Nota(peso=P4_peso, nota=5.0)
])==media_desejada)
)




SetimoCaso - Teste 1
Para as notas:
[ (Valor: 6.0, Peso: 0.12), (Valor: 6.0, Peso: 0.08), (Valor: 6.0, Peso: 0.08) ], pesos [0.12, 0.12, 0.18, 0.12, 0.18] e média 6 as notas possíveis são:
[ (1.5, 0.0, 9.0, 9.0, 8.0), (1.5, 0.0, 10.0, 6.0, 9.0), (1.5, 0.0, 10.0, 7.5, 8.0) ]
 True

SetimoCaso - Teste 2
Para as notas:
[ (Valor: 7.0, Peso: 0.2), (Valor: 7.0, Peso: 0.2) ], pesos [0.3, 0.3] e média 7 as notas possíveis são:
[ (5.5, 8.5), (6.0, 8.0), (4.0, 10.0) ]
True


Funcionou. Agora será implementado as melhorias até agora feitas

## 8. Oitavo Caso
Supondo que uma matéria tenha P1, P2, P3 e P4, sendo que já foram dadas as notas da P1, T1 e T2, quero saber para quais notas P2, P3, P4, T3 e T4 o aluno conseguirá média 7.

Facilitadores:
1. nota possível de ser determinada

Novidade:
1. o programa deve aceitar qualquer tamanho de notas que devem ser determinadas, assim como qualquer tamanho de notas de input
2. o programa parará de rodar ao escolher 3 notas possíveis
3. a ordem da permutação será aleatória
4. o algorítmo deve escolher notas com, no máximo, 3 de distância entre as notas e a média aritimética entre elas
5. a média pode variar do valor escolhido para `0.04` abaixo do esperado (para aceitar aredondamentos). *Ex*: se a média desejada for `6.0`, a combinação aceita pode ter média final entre `5.96` e `6.0`.


In [171]:
class OitavoCaso(Solucionador):
    NOTAS_TOTAIS=3
    MIN_DIST=3

    @staticmethod
    def algoritmo(notas_que_tenho: List[Nota], peso_de_notas_que_quero: List[float], media_desejada: float) -> List[Nota]:
        notas_possiveis = list()
        dominio_de_possiveis_notas = list()
        for _ in range(len(peso_de_notas_que_quero)):
            dominio_de_uma_nota = dominio_de_notas.copy()
            random.shuffle(dominio_de_uma_nota)
            dominio_de_possiveis_notas.append(dominio_de_uma_nota)
        idx_possiveis_notas = [0 for _ in range(len(peso_de_notas_que_quero))]
        
        while(idx_possiveis_notas != [19 for _ in range(len(peso_de_notas_que_quero))]):
            possiveis_notas_que_quero = [Nota(peso=peso_de_notas_que_quero[i], nota=dominio_de_possiveis_notas[i][idx_possiveis_notas[i]]) for i in range(len(peso_de_notas_que_quero))]
            todas_as_notas = notas_que_tenho + possiveis_notas_que_quero
            media = Utils.media(todas_as_notas)

            if(abs(media - media_desejada) <= 0.04):
                if(Utils.distancia_entre_notas([nota.nota for nota in possiveis_notas_que_quero], SextoCaso.MIN_DIST)):
                    notas_possiveis.append(tuple([nota.nota for nota in possiveis_notas_que_quero]))
            
            if(len(notas_possiveis) == OitavoCaso.NOTAS_TOTAIS): return notas_possiveis

            idx_possiveis_notas[-1] += 1
            idx_possiveis_notas_invertido = idx_possiveis_notas.copy()[::-1]
            for idx in range(0, len(idx_possiveis_notas)-1):
                if(idx_possiveis_notas_invertido[idx]) == 20:
                    idx_possiveis_notas_invertido[idx] = 0
                    idx_possiveis_notas_invertido[idx+1] += 1
                else:
                    break
                
            idx_possiveis_notas = idx_possiveis_notas_invertido[::-1]

        return notas_possiveis
    
    @staticmethod
    def teste_algoritmo(notas_que_tenho: Dict[float, float], peso_de_notas_que_quero: List[float], media_desejada: float) -> None:
        notas_possiveis = OitavoCaso.algoritmo(notas_que_tenho=notas_que_tenho, peso_de_notas_que_quero=peso_de_notas_que_quero, media_desejada=media_desejada)
        print("Para as notas:") 
        Utils.print_lista_de_notas(notas_que_tenho)
        print(f", pesos {peso_de_notas_que_quero} e média {media_desejada} as notas possíveis são:")
        Utils.print_lista_de_notas(notas_possiveis)
        print        

In [174]:
# Teste 1
print("\nOitavoCaso - Teste 1")
P1 = Nota(peso=0.2*0.6, nota=6.0)
T1 = Nota(peso=0.08, nota=6.0)

P2_peso = 0.2*0.6
T2 = Nota(peso=0.08, nota=6.0)

P3_peso = 0.3*0.6
T3_peso = 0.3*0.4

P4_peso = 0.3*0.6
T4_peso = 0.3*0.4

media_desejada = 6

l_notas_que_tenho = [P1, T1, T2]

OitavoCaso.teste_algoritmo(notas_que_tenho=l_notas_que_tenho, peso_de_notas_que_quero=[P2_peso, T3_peso, P3_peso, T4_peso, P4_peso], media_desejada=media_desejada)

# Verificação
notas = [
    P1,
    T1,
    T2,
    Nota(peso=P2_peso, nota=5),
    Nota(peso=T3_peso, nota=3),
    Nota(peso=P3_peso, nota=4.5),
    Nota(peso=T4_peso, nota=8.5),
    Nota(peso=P4_peso, nota=8.5)
]

print("\n",Utils.media(notas) == 6.0)

# Teste 2
print("\nOitavoCaso - Teste 2")
P1 = Nota(peso=0.2, nota=6.0)
P2 = Nota(peso=0.2, nota=8.0)
P3_peso = 0.3
P4_peso = 0.3
media_desejada = 7

l_notas_que_tenho = [P1, P2]

OitavoCaso.teste_algoritmo(notas_que_tenho=l_notas_que_tenho, peso_de_notas_que_quero=[P3_peso, P4_peso], media_desejada=media_desejada)

# Verificação
print("\n"+
str(Utils.media([
    P1,
    P2,
    Nota(peso=P3_peso, nota=9.0),
    Nota(peso=P4_peso, nota=5.0)
])==media_desejada)
)




OitavoCaso - Teste 1
Para as notas:
[ (Valor: 6.0, Peso: 0.12), (Valor: 6.0, Peso: 0.08), (Valor: 6.0, Peso: 0.08) ], pesos [0.12, 0.12, 0.18, 0.12, 0.18] e média 6 as notas possíveis são:
[ (6.5, 4.0, 6.5, 4.5, 7.5), (6.5, 4.0, 6.5, 6.5, 6.0), (6.5, 4.0, 6.5, 5.5, 7.0) ]
 True

OitavoCaso - Teste 2
Para as notas:
[ (Valor: 6.0, Peso: 0.2), (Valor: 8.0, Peso: 0.2) ], pesos [0.3, 0.3] e média 7 as notas possíveis são:
[ (8.5, 5.5), (9.5, 4.5), (7.0, 7.0) ]
True
